In [1]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, RNN, Lambda, Concatenate, Reshape, Dropout
from keras.layers.merge import Add
from keras.layers.merge import Multiply
from keras.models import Model
from keras.losses import mean_squared_error, binary_crossentropy
from keras.models import Model
from keras.models import load_model
from keras.models import model_from_json
import keras.backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
from Generators import pv_wb_generator

Using TensorFlow backend.


In [2]:
def save_model(model, model_str):
    model_json = model.to_json()
    with open('trained/'+model_str+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('trained/'+model_str+".h5")
    print("Saved model to disk")

In [3]:
def gen(batch_size=128):
    while True:
        a = np.zeros((batch_size,3)) #x, dx, v
        b = np.zeros((batch_size,3 )) #x, dx, v2
        if np.random.rand() > 0.25:
            a[:,2] = np.random.uniform(-0.1, 0.1, size=batch_size) #v
            a[:,0] = np.random.uniform(0.2, 0.8, size=batch_size) #x
            a[:,1] = 0
            
            b[:,2] = a[:,2]
            b[:,0] = a[:,0]+a[:,2]
            b[:,1] = 0
            yield a, b
        if np.random.rand() > 0.5:
            #border at 1
            a[:,2] = np.random.uniform(0, 0.1, size=batch_size) #v
            a[:,0] = np.random.uniform(1-a[:,2]*2, 1, size=batch_size) #x
            a[:,1] = np.maximum(a[:,2] - (1-a[:,0]), 0) #dx
            
            indecesb = np.argwhere(a[:,1] > 0) 
            indecess = np.argwhere(a[:,1] == 0)
            b[:,2] = a[:,2]
            b[indecesb,2] *= -1 #-v
            b[indecess,0] = a[indecess,0] + a[indecess,2]
            b[indecesb,0] = 1-a[indecesb,1]
            b[:,1] = np.maximum(b[:,2] - (1-b[:,0]), 0)
            yield a, b
        else:
            #border at 0
            a[:,2] = np.random.uniform(0, -0.1, size=batch_size) #v
            a[:,0] = np.random.uniform(-a[:,2]*2, 0, size=batch_size) #x
            a[:,1] = np.maximum(-a[:,2] - a[:,0], 0) #dx
            
            indecesb = np.argwhere(a[:,1] > 0) 
            indecess = np.argwhere(a[:,1] == 0)
            b[:,2] = a[:,2]
            b[indecesb,2] *= -1 #-v
            b[indecess,0] = a[indecess,0] + a[indecess,2]
            b[indecesb,0] = a[indecesb,1]
            b[:,1] = np.maximum(-b[:,2] - b[:,0], 0)
            yield a, b

In [4]:
def gen(batch_size=128):
    while True:
        a = np.zeros((batch_size,3)) #x, dx, v
        b = np.zeros((batch_size,3 )) #x, dx, v2
        i = np.random.choice([0,0,1,2], size=batch_size)
#         i = np.random.choice([2], size=batch_size)

        i1 = np.argwhere(i==0)
        i1 = i1.reshape(-1,1)[:,0]
#         print('i1: ', i1)
        a[i1,2] = np.random.uniform(-0.1, 0.1, size=i1.shape[0]) #v
        a[i1,0] = np.random.uniform(0.2, 0.8, size=i1.shape[0]) #x
        a[i1,1] = 0

        b[i1,2] = a[i1,2]
        b[i1,0] = a[i1,0]+a[i1,2]
        b[i1,1] = 0

        #border at 1
        i2 = np.argwhere(i==1)
        i2 = i2.reshape(-1,1)[:,0]
#         print('i2: ', i2)
        a[i2,2] = np.random.uniform(0, 0.1, size=i2.shape[0]) #v
        a[i2,0] = np.random.uniform(1-a[i2,2]*2, 1, size=i2.shape[0]) #x
        a[i2,1] = np.maximum(a[i2,2] - (1-a[i2,0]), 0) #dx

        indecesb = np.argwhere(a[i2,1] > 0) 
        indecess = np.argwhere(a[i2,1] == 0)
        b[i2,2] = a[i2,2]
        b[i2[indecesb],2] *= -1 #-v
        b[i2[indecess],0] = a[i2[indecess],0] + a[i2[indecess],2]
        b[i2[indecesb],0] = 1-a[i2[indecesb],1]
        b[i2,1] = np.maximum(b[i2,2] - (1-b[i2,0]), 0)

        #border at 0
        i3 = np.argwhere(i==2)
        i3 = i3.reshape(-1,1)[:,0]
#         print('i3: ', i3)
        a[i3,2] = np.random.uniform(0, -0.1, size=i3.shape[0]) #v
        a[i3,0] = np.random.uniform(-a[i3,2]*2, 0, size=i3.shape[0]) #x
        a[i3,1] = np.maximum(-a[i3,2] - a[i3,0], 0) #dx

        indecesb = np.argwhere(a[i3,1] > 0) 
        indecess = np.argwhere(a[i3,1] == 0)
        b[i3,2] = a[i3,2]
        b[i3[indecesb],2] *= -1 #-v
        b[i3[indecess],0] = a[i3[indecess],0] + a[i3[indecess],2]
        b[i3[indecesb],0] = a[i3[indecesb],1]
        b[i3,1] = np.maximum(-b[i3,2] - b[i3,0], 0)
        yield a, b

In [5]:
a,b = next(gen(batch_size=5))

In [6]:
a,b

(array([[ 0.990098  ,  0.        ,  0.00868419],
        [ 0.60491118,  0.        ,  0.08044642],
        [ 0.01470408,  0.01403718, -0.02874126],
        [ 0.41049072,  0.        , -0.03102929],
        [ 0.85672816,  0.        ,  0.09671656]]),
 array([[ 0.99878218,  0.00746637,  0.00868419],
        [ 0.6853576 ,  0.        ,  0.08044642],
        [ 0.01403718,  0.        ,  0.02874126],
        [ 0.37946143,  0.        , -0.03102929],
        [ 0.95344472,  0.05016127,  0.09671656]]))

In [7]:
from keras.regularizers import l2

In [8]:
inputs = Input(shape=(3,))

split_dv = Lambda(lambda s: s[:,1:], output_shape=(2,))(inputs)
h1 = Dense(4, activation='relu', kernel_regularizer=l2(0.01))(split_dv)
# h1 = Dense(8, activation='relu')(h1)

split_xd = Lambda(lambda s: s[:,:2], output_shape=(2,))(inputs)
h2 = Dense(4, activation='relu', kernel_regularizer=l2(0.01))(split_xd)
h2 = Dense(8, activation='relu', kernel_regularizer=l2(0.01))(h2)

h = Concatenate()([h1,h2])
h = Dense(8, activation='relu', kernel_regularizer=l2(0.01))(h)
# h = Dense(64, activation='relu')(h)
h = Dense(3, )(h)

preds = h
module = Model(inputs=inputs, outputs=preds)
module.compile(loss=mean_squared_error,
              optimizer='adam')
module.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3)            0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 2)            0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 2)            0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 4)            12          lambda_2[0][0]                   
__________________________________________________________________________________________________
dense_1 (D

In [9]:
# inputs = Input(shape=(3,))

# split_dv = Lambda(lambda s: s[:,:3], output_shape=(3,))(inputs)


# split_xd = Lambda(lambda s: s[:,:3], output_shape=(3,))(inputs)

# # h = Dense(32)(inputs)
# h = Dense(8, activation='relu')(inputs)
# h = Dense(128, activation='relu')(h)
# h = Dense(3)(h)

# preds = h

# module = Model(inputs=inputs, outputs=preds)
# module.compile(loss=mean_squared_error,
#               optimizer='adam')
# module.summary()

In [10]:
try:
    module.fit_generator(gen(batch_size=1280), steps_per_epoch=128000)
except KeyboardInterrupt:
    pass

Epoch 1/1
 57754/128000 [============>.................] - ETA: 2:02 - loss: 0.0073

50098/128000 [==========>...................] - ETA: 2:25 - loss: 0.0125

In [10]:
X, Y = next(gen(batch_size=1280))
P = module.predict(X)
mse(P, Y)**0.5

0.005302176242641814

In [145]:
X, Y = next(gen(batch_size=1280))
P = module.predict(X)
mse(P, Y)**0.5

0.0009170507644204933

In [11]:
save_model(module, 'module')

Saved model to disk


In [60]:
X[:10]

array([[ 3.95293876e-01,  0.00000000e+00, -2.31737371e-02],
       [ 5.69822649e-02,  0.00000000e+00, -3.38937842e-02],
       [ 7.12937147e-01,  0.00000000e+00, -2.80543810e-03],
       [ 2.50752071e-01,  0.00000000e+00, -8.35852589e-02],
       [ 5.54419064e-01,  0.00000000e+00,  6.02066112e-02],
       [ 6.68203812e-02,  0.00000000e+00, -4.88965262e-02],
       [ 1.27540723e-02,  4.58387632e-03, -1.73379486e-02],
       [ 9.49308778e-01,  0.00000000e+00,  2.66804224e-02],
       [ 9.65875666e-01,  5.12371971e-04,  3.46367058e-02],
       [ 2.41085456e-01,  0.00000000e+00, -6.52882094e-02]])

In [61]:
P[:10]

array([[ 3.7211600e-01,  3.4725666e-04, -2.2776678e-02],
       [ 2.3001421e-02,  1.0808498e-02, -3.4335718e-02],
       [ 7.1012193e-01, -1.2615323e-04, -2.6649833e-03],
       [ 1.6710773e-01,  1.0934472e-04, -8.3116539e-02],
       [ 6.1471426e-01,  6.0459971e-04,  6.0697332e-02],
       [ 1.7910000e-02,  3.0890554e-02, -4.9299031e-02],
       [ 4.7469325e-03, -4.7236681e-05,  1.7666474e-02],
       [ 9.7613239e-01,  3.5030544e-03,  2.7071312e-02],
       [ 9.9962044e-01,  1.2782216e-04, -3.4717202e-02],
       [ 1.7573161e-01,  1.6060472e-04, -6.4848855e-02]], dtype=float32)

In [62]:
Y[:10]

array([[ 0.37212014,  0.        , -0.02317374],
       [ 0.02308848,  0.0108053 , -0.03389378],
       [ 0.71013171,  0.        , -0.00280544],
       [ 0.16716681,  0.        , -0.08358526],
       [ 0.61462567,  0.        ,  0.06020661],
       [ 0.01792386,  0.03097267, -0.04889653],
       [ 0.00458388,  0.        ,  0.01733795],
       [ 0.9759892 ,  0.00266962,  0.02668042],
       [ 0.99948763,  0.        , -0.03463671],
       [ 0.17579725,  0.        , -0.06528821]])

In [64]:
save_model(module, 'module_939params_200k')

Saved model to disk
